
## Prophet Modeling with Multiple Regressors

This analysis incorporates additional regressors (e.g., precipitation and rolling temperature averages) to improve forecast accuracy and capture complex relationships in the climate data.


In [1]:

# Prepare data for Prophet with multiple regressors
temperature_data = data[['date', 'avg_temperature_v', 'precip_rolling_avg', 'temp_rolling_avg']].rename(columns={
    'date': 'ds',
    'avg_temperature_v': 'y',
    'precip_rolling_avg': 'precip',
    'temp_rolling_avg': 'temp_rolling'
})

# Ensure no NaN values in the regressors
temperature_data['precip'] = temperature_data['precip'].fillna(method='ffill').fillna(method='bfill')
temperature_data['temp_rolling'] = temperature_data['temp_rolling'].fillna(method='ffill').fillna(method='bfill')

# Initialize and fit Prophet model with multiple regressors
multi_regressor_model = Prophet(yearly_seasonality=True)
multi_regressor_model.add_regressor('precip')
multi_regressor_model.add_regressor('temp_rolling')

# Fit the model
multi_regressor_model.fit(temperature_data)

# Forecast with future dataframe including regressors
future = multi_regressor_model.make_future_dataframe(periods=60, freq='M')
future['precip'] = temperature_data['precip'].iloc[-1]  # Use latest known value for precipitation
future['temp_rolling'] = temperature_data['temp_rolling'].iloc[-1]  # Use latest known value for rolling temperature
multi_forecast = multi_regressor_model.predict(future)

# Plot forecast
fig1 = multi_regressor_model.plot(multi_forecast)
plt.title('Forecast with Multiple Regressors (Precipitation and Rolling Temperature)')
plt.show()

# Plot components
fig2 = multi_regressor_model.plot_components(multi_forecast)
plt.show()


NameError: name 'data' is not defined


## Advanced Analysis with Multiple Regressors

Building on the multiple regressor model, this section includes changepoint analysis, scenario exploration, residual diagnostics, and confidence interval evaluation.


In [ ]:

# Changepoint Analysis
changepoints = multi_regressor_model.changepoints
changepoint_effects = multi_forecast.loc[multi_forecast['ds'].isin(changepoints), ['ds', 'trend']]

# Visualize changepoint effects
plt.figure(figsize=(12, 6))
plt.plot(multi_forecast['ds'], multi_forecast['trend'], label='Trend', color='blue')
plt.scatter(changepoint_effects['ds'], changepoint_effects['trend'], color='red', label='Changepoints')
plt.title('Changepoint Effects on Trend (Multiple Regressors)')
plt.xlabel('Date')
plt.ylabel('Trend')
plt.legend()
plt.show()


In [ ]:

# Scenario Analysis: Modify regressors
scenario_future = future.copy()
scenario_future['precip'] += 10  # Increase precipitation trend by 10 units

# Forecast under the scenario
scenario_forecast = multi_regressor_model.predict(scenario_future)

# Plot scenario vs. original forecast
plt.figure(figsize=(12, 6))
plt.plot(multi_forecast['ds'], multi_forecast['yhat'], label='Original Forecast', color='blue')
plt.plot(scenario_forecast['ds'], scenario_forecast['yhat'], label='Scenario: Increased Precipitation', color='green')
plt.title('Scenario Analysis: Increased Precipitation Impact on Temperature')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()


In [ ]:

# Residual Analysis for Multiple Regressors
multi_forecast['residuals'] = multi_forecast['yhat'][:len(temperature_data)] - temperature_data['y']

# Plot residuals
plt.figure(figsize=(12, 6))
plt.plot(temperature_data['ds'], multi_forecast['residuals'][:len(temperature_data)], label='Residuals')
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residuals from Multiple Regressor Prophet Model')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.legend()
plt.show()

# Residual diagnostics
residuals = multi_forecast['residuals'].dropna()
stat, p_value = normaltest(residuals)
dw_stat = durbin_watson(residuals)
print(f"Normality Test: p-value = {p_value} (p > 0.05 indicates normality)")
print(f"Durbin-Watson Statistic: {dw_stat} (close to 2 indicates no autocorrelation)")


In [ ]:

# Confidence Interval Analysis
plt.figure(figsize=(12, 6))
plt.fill_between(multi_forecast['ds'], multi_forecast['yhat_lower'], multi_forecast['yhat_upper'], color='skyblue', alpha=0.4, label='Confidence Interval')
plt.plot(multi_forecast['ds'], multi_forecast['yhat'], color='blue', label='Forecast')
plt.plot(temperature_data['ds'], temperature_data['y'], color='black', label='Observed')
plt.title('Forecast with Confidence Intervals (Multiple Regressors)')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()
